In [29]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
auth.authenticate_user()
print('Authenticated')

project_id = 'alridho-hacktive8'
client = bigquery.Client(project=project_id)

Authenticated


In [38]:
df = client.query('''SELECT * FROM `ftds-hacktiv8-project.practice.bike_rent`''').to_dataframe()
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60
...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             8000 non-null   object             
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(2), datetime64[us, UTC](1), object(5)
memory usage: 515.8+ KB


# SQL

1. Berapa banyak label unique dari `bike_type`?

In [40]:
df_1 = client.query('''

SELECT COUNT(DISTINCT bike_type) AS Banyak_Label_Unique
FROM `ftds-hacktiv8-project.practice.bike_rent`

''').to_dataframe()

df_1

,Banyak_Label_Unique
0,2


Banyak label unique dari column bike_type adalah `2`

2. Apa saja nama label unique dari tipe sepeda?

In [41]:
df_1 = client.query('''

SELECT DISTINCT bike_type AS Label_Unique
FROM `ftds-hacktiv8-project.practice.bike_rent`

''').to_dataframe()

df_1

,Label_Unique
0,classic
1,electric


Label unique dari column bike_type adalah classic dan electric

3. Berapa banyak total peminjaman sepeda pada tahun 2016 dan 2017?

In [50]:
df_1 = client.query('''

SELECT
  SUM(CASE WHEN EXTRACT(YEAR FROM start_time) = 2016 THEN 1 ELSE 0 END) AS total_2016,
  SUM(CASE WHEN EXTRACT(YEAR FROM start_time) = 2017 THEN 1 ELSE 0 END) AS total_2017,
  SUM(CASE WHEN EXTRACT(YEAR FROM start_time) IN (2016, 2017) THEN 1 ELSE 0 END) AS total_both_years
FROM `ftds-hacktiv8-project.practice.bike_rent`;

''').to_dataframe()

df_1

,total_2016,total_2017,total_both_years
0,3454,4539,7993


Total peminjaman sepeda pada tahun 2016 dan 2017 adalah `7993`

4. Dimana station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016?

In [66]:
df_1 = client.query('''

SELECT start_station_name, COUNT(*) AS total_departures
FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE EXTRACT(YEAR FROM start_time) IN (2016)
GROUP BY start_station_name
ORDER BY total_departures DESC
LIMIT 25;

''').to_dataframe()

df_1


,start_station_name,total_departures
0,Bullock Museum @ Congress & MLK,394
1,Davis at Rainey Street,289
2,East 11th St. & San Marcos,286
3,Trinity & 6th Street,234
4,State Capitol @ 14th & Colorado,205
5,Palmer Auditorium,196
6,ACC - West & 12th Street,185
7,East 11th St. at Victory Grill,176
8,11th & San Jacinto,171
9,Pfluger Bridge @ W 2nd Street,162


Station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016 adalah `Bullock Museum @ Congress & MLK`

5. Berapa nilai minimum, maximum, dan rata-rata dari durasi perjalanan pada keseluruhan data?

In [52]:
df_1 = client.query('''

SELECT
    MIN(duration_minutes) AS min_duration,
    MAX(duration_minutes) AS max_duration,
    AVG(duration_minutes) AS avg_duration
FROM
    `ftds-hacktiv8-project.practice.bike_rent`;

''').to_dataframe()

df_1


,min_duration,max_duration,avg_duration
0,2,2218,35.0995


* Nilai minimum durasi adalah 2 menit
* Nilai maximum durasi adalah 2218 menit
* Nilai rata-rata durasi adalah 35.0995 menit

6. Berapa rata-rata durasi perjalanan dari subscriber_type Explorer dan Weekender?

In [77]:
df_1 = client.query('''

SELECT subscriber_type, AVG(duration_minutes) AS avg_duration
FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE subscriber_type IN ('Explorer','Weekender')
GROUP BY subscriber_type;

''').to_dataframe()

df_1

,subscriber_type,avg_duration
0,Explorer,42.866667
1,Weekender,27.636478


* Rata-rata durasi perjalanan dari subscriber_type Explorer adalah `42.866667`
* Rata-rata durasi perjalanan dari subscriber_type Weekender adalah `27.636478`

7. Tampilkan entry data yang memiliki duration_time diatas 4.25 jam. (Gunakan subquery untuk menjawab pertanyaan ini dengan meng-convert duration_time dari menit ke hour).

In [95]:
df_1 = client.query('''

# SELECT * FROM `ftds-hacktiv8-project.practice.bike_rent`
# WHERE duration_minutes > 255                                    #Cara Non-Subquery
# ORDER BY duration_minutes ASC;

SELECT * FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE
      (SELECT duration_minutes / 60.0) > 4.25                   #Cara Subquery
ORDER BY duration_minutes ASC;

''').to_dataframe()

df_1

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,9576501,Walk Up,203,classic,2016-04-15 13:25:33+00:00,East 11th St. at Victory Grill,East 11th St. at Victory Grill,257
1,9576454,Walk Up,550,classic,2016-04-15 13:23:32+00:00,East 11th St. at Victory Grill,East 11th St. at Victory Grill,258
2,12821266,Walk Up,54,classic,2016-11-14 09:59:39+00:00,San Jacinto & 8th Street,11th & San Jacinto,260
3,12821253,Walk Up,867,classic,2016-11-14 09:58:03+00:00,San Jacinto & 8th Street,11th & San Jacinto,261
4,12821259,Walk Up,25,classic,2016-11-14 09:58:47+00:00,San Jacinto & 8th Street,11th & San Jacinto,261
...,...,...,...,...,...,...,...,...
86,10197780,Walk Up,354,classic,2016-05-28 03:34:42+00:00,Lavaca & 6th,Lavaca & 6th,1064
87,10047507,Weekender,97,classic,2016-05-18 15:12:37+00:00,East 7th & Pleasant Valley,Lavaca & 6th,1126
88,15912175,Walk Up,663,classic,2017-09-10 16:08:17+00:00,Henderson & 9th,Henderson & 9th,1160
89,10087505,Walk Up,895,classic,2016-05-21 01:20:46+00:00,East 11th St. at Victory Grill,East 11th St. at Victory Grill,2113


Berikut adalah seluruh data dengan duration_minutes > dari 4.25 jam

# PANDAS

In [97]:
import pandas as pd
df = client.query('''SELECT * FROM `ftds-hacktiv8-project.practice.bike_rent`''').to_dataframe()
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60
...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55


1. Cek kesesuaian tipe data dan keberadaan missing values dengan `.info()`

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             8000 non-null   object             
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(2), datetime64[us, UTC](1), object(5)
memory usage: 515.8+ KB


Data sudah sesuai dan tidak ditemukan missing values

2. Buat kolom baru bernama quarter.

In [103]:
df['start_time'] = pd.to_datetime(df['start_time'])

q = []
for bulan in df['start_time'].dt.month:
    if bulan in [1,2,3]:
        q.append('Q1')
    elif bulan in [4,5,6]:
        q.append('Q2')
    elif bulan in [7,8,9]:
        q.append('Q3')
    else:
        q.append('Q4')

df['Quarter'] = q
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,Quarter
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178,Q4
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,Q1
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60,Q4
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q4
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q4
...,...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55,Q3
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55,Q4
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q1
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q2


Berikut adalah hasil dari membuat kolom Quarter

3. Tampilkan data perjalanan dari tipe member `Local365` dengan durasi perjalanan diatas 1 jam `(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)`.

In [106]:
data_perjalanan = df[(df['subscriber_type'] == 'Local365') & (df['duration_minutes'] > 60)]
data_perjalanan

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,Quarter
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,Q1
94,15118837,Local365,693,classic,2017-06-14 12:48:06+00:00,Lavaca & 6th,Lavaca & 6th,81,Q2
165,12594741,Local365,332,classic,2016-10-28 11:13:44+00:00,Palmer Auditorium,Lavaca & 6th,147,Q4
359,14489810,Local365,955,classic,2017-04-18 08:02:59+00:00,11th & San Jacinto,11th & San Jacinto,134,Q2
401,18839331,Local365,343,classic,2018-10-19 19:43:22+00:00,Davis at Rainey Street,11th & San Jacinto,823,Q4
428,14041702,Local365,27,classic,2017-03-15 08:26:51+00:00,13th & San Antonio,13th & San Antonio,123,Q1
453,15862398,Local365,395,classic,2017-09-05 11:47:16+00:00,13th & San Antonio,13th & San Antonio,77,Q3
608,16520009,Local365,518,classic,2017-12-18 09:03:12+00:00,Palmer Auditorium,Congress & Cesar Chavez,290,Q4
609,15886237,Local365,862,classic,2017-09-08 08:07:12+00:00,South Congress & Academy,Congress & Cesar Chavez,269,Q3
704,9794912,Local365,30,classic,2016-05-01 10:50:37+00:00,South Congress & Academy,Barton Springs & Riverside,113,Q2


Berikut adalah data perjalanan dari tipe member Local365 dengan durasi perjalanan diatas 1 jam

4. Berapa rata-rata durasi perjalanan dari tipe pelanggan `Walk Up` dengan tipe sepeda `classic` pada bulan `Maret` berdasarkan data yang diambil? `(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)`.

In [118]:
rata_rata_durasi = df[(df['subscriber_type'] == 'Walk Up') & (df['bike_type'] == 'classic') & (df['start_time'].dt.month == 3)]
average_duration = rata_rata_durasi['duration_minutes'].mean()
average_duration
# rata_rata_durasi  # Untuk menampilkan tabel

38.476584022038566

Rata-rata durasi perjalanan dari tipe pelanggan Walk Up dengan tipe sepeda classic pada bulan Maret berdasarkan data yang diambil adalah 38.476584022038566